<a href="https://colab.research.google.com/github/Momir94/Analytics/blob/main/DCF_Valuation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install ipywidgets

In [ ]:
import requests
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import bs4
from pycoingecko import CoinGeckoAPI
from collections import Counter
import matplotlib as mpl
from scipy.optimize import minimize
from scipy.optimize import Bounds
import crv
import uni

cg = CoinGeckoAPI()

pd.options.mode.chained_assignment = None  # default='warn' exclude warning for chained assingments

In [ ]:
pd.set_option('display.float_format', lambda x: '%.4f' % x) ## change float at the macro level
plt.rcParams["figure.figsize"] = (14, 6) ## change plot size at the macro level
mpl.rcParams['figure.dpi'] = 300 ## graph resolution

## Model inputs

1. number of years prior to calculating terminal growth (default 10)
2. growth rates over the given number of years (default assume decreasing growth rate starting with the highest in year 1 and decreasing at a certain ***factor*** till reaching terminal growth rate)
3. terminal growth rate
4. discount rate (we assume discount rates are not static, but also decrease with the time)
5. monthly base (total trading volume, total borrowing volume, etc.)
6. fees; where base * fees = revenue
7. token distribution

In [ ]:
num_years = 10 #number of years prior to calculating terminal growth
num_periods = num_years * 12 #10 years monthly data (assuming each month has 30 days)


def dates(num_periods = num_periods):    
    start = pd.to_datetime(dt.datetime.today().date())
    starts = [start]
    for i in range(num_periods - 1):
        starts.append(starts[i] + dt.timedelta(30))
        
    return starts

starts = dates()

## Token Daily Details (Unique)
Would likely require manual work in collecting and sometimes approximating token distribution. 

Regardless of the different token make sure that:
- Token Distribution is stored as pandas Series, 
- Daily data
- Start date of the Series should be before (equal) the analysis start date, while the end date should be after (equal) to the analysis end date. 

The formula will automatically exclude irrelevant values. For instance, if analysis is covering monthly intervals, it will filter only one day per month. This is approximation, to futher improve precision we should consider averaging number of tokens for each month rather than assuming that number during the month was constant and equal to the particular date. 


In [ ]:
Token_distribution = crv.token_distribution(starts)

True


C:\Users\amidz\Desktop\Quandefi\crv.py:28: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  CRV_distribution = CRV_distribution.append(pd.Series(index = dates, data = crvs))


## Choosing the right decreasing schedule to give us reasonable growth and discount rate in year 10

- make sure that the discount rate in the final year of estimation is larger than the terminal growth rate

In [ ]:
def terminal (expansion_growth, decreasing_factor, num_years = 10):
    se_ = (np.ones(num_years))
    se_[0] = expansion_growth
    se_[1:] = decreasing_factor

    growth_rates = list(np.cumprod(se_))
    
    return growth_rates[-1]


def compute_decr_r (target_terminal, guess, expansion_growth):

    '''''''''
    Computing decreasing rate factor to satisfy target terminal rate knowing that the expected growth rate in the first year is equal to certain number.
    
    target_terminal - target growth/discount rate in year 10 e.g. 15%
    guess - guess the decreasing rate e.g. 0.8
    expansion_growth - first year growth/discount rate
    '''''''''
    
    x = guess 
    f = lambda x: terminal(expansion_growth, decreasing_factor=x) #function to minimize 
    cons = [{"type": "eq", "fun": lambda x: terminal(expansion_growth, decreasing_factor=x) - target_terminal}] 
    res = minimize(f, x, method='SLSQP', constraints=cons)   

    return res.x[0]

In [ ]:
terminal_growth = 0.05 #ANNUAL
expansion_growth = 0.5

decreasing_factor = compute_decr_r (terminal_growth, 0.8, expansion_growth)

def schedule (decreasing_factor, year1):
    schedule = np.ones(10) * decreasing_factor
    schedule[0] = year1
    rates = pd.DataFrame(np.cumprod(schedule), index = np.arange(1,11), columns=['Rates'])
    
    return rates

growth_rates = schedule(decreasing_factor, expansion_growth)
growth_rates

,Rates
1,0.5000
2,0.3871
3,0.2997
4,0.2321
5,0.1797
6,0.1391
7,0.1077
8,0.0834
9,0.0646
10,0.0500


In [ ]:
discount = 0.3769 #annual
decreasing_factor_discount = compute_decr_r (0.15, 0.7, discount)

discount_rates = schedule(decreasing_factor_discount, discount)
discount_rates

,Rates
1,0.3769
2,0.3402
3,0.3071
4,0.2772
5,0.2502
6,0.2259
7,0.2039
8,0.1840
9,0.1661
10,0.1500


## Model parameter set-up

In [ ]:
staked = 1 #Some protocols require tokens to be staked in order to capture revenue, we approach valuation from a staker perspective. If there is no staking requirement or everyone stakes we can simply set this parameter equal to 1. For simplicity assume always 1, otherwise we also have to change discount rate to account for illiquidity. 
base = 115000000 * 30 # $ Volume monthly
fees = 0.0002 # Protocol fees in case volume is basis / Interest rate in case TVL is basis

In [ ]:
def DCF_simple(growth_rates = growth_rates, discount_rates = discount_rates, terminal_growth_in_profit = terminal_growth,
    num_years = num_years, num_periods = num_periods, starts = starts, Token_distribution = Token_distribution, 
               base = base, rates = fees, tokens_staked = staked):
    
    growth_rates = list(growth_rates['Rates'])
    discount_rates = list(discount_rates['Rates'])
    
    df = pd.DataFrame(data = np.arange(1, num_periods+1), index = starts, columns=["Index"])
    df["Monthly Growth Rates"] = 1

    t = 0
    for i in range(num_years):
        df["Monthly Growth Rates"][t:t+12] = (1+growth_rates[i])**(1/12)-1
        t+=12

        
    df["Monthly Discount Rates"] = 1

    t = 0
    for i in range(num_years):
        df["Monthly Discount Rates"][t:t+12] = (1+discount_rates[i])**(1/12)-1
        t+=12
  
        
    df = pd.concat([df, pd.DataFrame(Token_distribution[starts], columns=["Token Distribution"])], axis = 1) 
    df["Staked Tokens"] = tokens_staked * df["Token Distribution"]

    ## Expected Volume

    df["Forecast"] = 1
    forecast = [base*(1+growth_rates[0])**(1/12)]
    for i in range(len(df)-1):
        forecast.append(forecast[i]*(1+list(df["Monthly Growth Rates"])[i+1]))

    df["Forecast"] = forecast

    ## REVENUE
    df["Protocol Revenue"] = df["Forecast"] * rates
    df["Revenue Per Token"] = df["Protocol Revenue"] / df["Staked Tokens"] ## Revenue per staked tokens

    ##DISCOUNTING 
    df["Discount Factor"] = np.cumprod(df['Monthly Discount Rates']+1)
    df["Discounted Revenue Per Token"] = df["Revenue Per Token"] / df["Discount Factor"]

    #TERMINAL VALUE
    
    terminal_growth_monthly = (1+terminal_growth_in_profit)**(1/12)-1
    terminal_discount_monthly = df['Monthly Discount Rates'][-1]
    terminal_discount = (1+terminal_discount_monthly)**12 - 1
    
    TV = df["Revenue Per Token"][-1] * (1+terminal_growth_monthly) / (terminal_discount_monthly - terminal_growth_monthly)
    TV_discounted = TV / df['Discount Factor'][-1]

    #FINAL PRICE
    price_per_token = TV_discounted + df["Discounted Revenue Per Token"].sum()

    return price_per_token

In [ ]:
DCF_simple()

0.05622402346726865

In [ ]:
def implied_growth (current_price, x = 0.8):
    '''''''''        
    current_price - token price to be used as a basis for deriving implied growth rates.
    
    x - initial guess for the optimization problem by default 100%.
    '''''''''
    
    f = lambda x: DCF_simple(growth_rates = schedule(decreasing_factor, x)) #function to minimize 
    cons = [{"type": "eq", "fun": lambda x: DCF_simple(growth_rates = schedule(decreasing_factor, x)) - current_price}] 
    res = minimize(f, x, method='SLSQP', constraints=cons)   
    
    return res.x[0]

In [ ]:
implied_growth(current_price=1)

1.8630717603108418

In [ ]:
annual_base = base * 12
target_multiple = 15

In [ ]:
def multiples(target_multiple = target_multiple, growth_rates = growth_rates, discount_rates = discount_rates, 
              num_years = num_years, Token_distribution = Token_distribution, 
               base = annual_base, rates = fees, annual_discount = discount):
    
    
    growth_rates = list(growth_rates['Rates'])[:num_years]
    discount_rates = list(discount_rates['Rates'])[:num_years]


    df = pd.DataFrame(data = np.arange(1, num_years+1), index = np.arange(1, num_years+1), columns=["Year"])
    df["Annual Growth Rates"] = 1


    for i in range(num_years):
        df["Annual Growth Rates"][i:i+1] = (1+growth_rates[i])


    ## Expected Volume

    df["Forecast"] = 1

    forecast = [annual_base*(1+growth_rates[0])]
    for i in range(len(df)-1):
        forecast.append(forecast[i]*(list(df["Annual Growth Rates"])[i+1]))

    df["Forecast"] = forecast

    ## REVENUE
    df["Protocol Revenue"] = df["Forecast"] * rates
    
    market_cap = target_multiple * df.iloc[-1, -1]
    
    df["Target Multiple"] = target_multiple 
    
    df["Market Cap"] = df["Target Multiple"] * df["Protocol Revenue"]
    
    
    
    dates = []
    for i in range(num_years):
        dates.append(dt.datetime((dt.datetime.today() + dt.timedelta(365*(i+1))).year, 
                (dt.datetime.today() + dt.timedelta(365*(i+1))).month, 
                (dt.datetime.today() + dt.timedelta(365*(i+1))).day))
        
    circ_supply = []
    for i in range(num_years):
        circ_supply.append(Token_distribution[Token_distribution.index == dates[i]][0])
        
    df["Circulating Supply"] = circ_supply
    
    df["Price Per Token"] = df["Market Cap"] / df["Circulating Supply"]
    
    df["Discount Rates"] = discount_rates
        

    date = dt.datetime((dt.datetime.today() + dt.timedelta(365*num_years)).year, 
                (dt.datetime.today() + dt.timedelta(365*num_years)).month, 
                (dt.datetime.today() + dt.timedelta(365*num_years)).day)


    circulating_supply = Token_distribution[Token_distribution.index == date][0]

    price_per_token = market_cap/circulating_supply
    
    discount_rates.reverse()

    for i in range(len(discount_rates)):
        discount_rates[i] = 1/(discount_rates[i]+1)

    discount_rates[0] = price_per_token * discount_rates[0]

    price = (np.cumprod(discount_rates))


    return price[-1]

In [ ]:
def implied_growth1 (current_price, num_years, guess = 1):
    '''''''''
    terminal growth - growth in profits into perpetuity after 10 years of dynamic growth rates.
    
    base - if exchange, then MONTHLY volume to be used as a base in the estimation.
    
    fees - if exchange, then fees that are basis for revenue calculation e.g. 
        in case of Curve it is 0.0002 despite Curve is charging 0.0004 per transaction as 
        fees are split 50:50 between protocol and liquidity providers. Simply, we aim to
        calculate protocol revenue only. 
    
    discount - annual discount rate
        
    current_price - tokens price to be used as a basis for deriving implied growth rates.
    
    guess - initial guess for the optimization problem by default 100%.
    '''''''''
        
    x = guess 
    f = lambda x: multiples(growth_rates = schedule(decreasing_factor, x), num_years = num_years) #function to minimize 
    cons = [{"type": "eq", "fun": lambda x: multiples(growth_rates = schedule(decreasing_factor, x), num_years=num_years) - current_price}] 
    res = minimize(f, x, method='SLSQP', constraints=cons)   
    
    return res.x[0]

In [ ]:
implied_growth1(current_price=1, num_years=9)

1.9391721959896806

# Sanity Check

In [ ]:
def sanity (current_price, num_years = 5):
    
    se_ = (np.ones(num_years))
    se_[0] = implied_growth(current_price=current_price)
    se_[1:] = decreasing_factor

    growth_rates = (np.cumprod(se_))    
    
    expected_volume = np.ones(len(growth_rates))
    expected_volume[0] = base * 12 * (1+growth_rates[0])

    for i in range(1, len(growth_rates)):
        expected_volume[i] = expected_volume[i-1] * (1+growth_rates[i])

    expected = pd.DataFrame([np.arange(dt.datetime.today().year+1, dt.datetime.today().year + num_years+1), 
                             expected_volume/1000000000/360, growth_rates * 100]).T    
    expected.columns = ['Year', 'Implied Daily Average Volume in $B', 'Growth Rates %']
    expected = expected.set_index('Year')
    
    return expected 

In [ ]:
sanity(1)

,Implied Daily Average Volume in $B,Growth Rates %
Year,,
2023.0000,0.3293,186.3072
2024.0000,0.8042,144.2509
2025.0000,1.7024,111.6882
2026.0000,3.1746,86.4761
2027.0000,5.3001,66.9553


In [ ]:
def sanity1 (current_price, num_years):
    
    se_ = (np.ones(num_years))
    se_[0] = implied_growth1(current_price=current_price, num_years=num_years)
    se_[1:] = decreasing_factor

    growth_rates = (np.cumprod(se_))    
    
    expected_volume = np.ones(len(growth_rates))
    expected_volume[0] = base * 12 * (1+growth_rates[0])

    for i in range(1, len(growth_rates)):
        expected_volume[i] = expected_volume[i-1] * (1+growth_rates[i])


    expected = pd.DataFrame([np.arange(dt.datetime.today().year+1, dt.datetime.today().year + num_years+1), expected_volume/1000000000/360, growth_rates * 100]).T
    expected.columns = ['Year', 'Implied Daily Average Volume in $B', 'Growth Rates %']
    expected = expected.set_index('Year')
    
    return expected 

In [ ]:
sanity1(1, 5)

,Implied Daily Average Volume in $B,Growth Rates %
Year,,
2023.0000,0.3698,221.6086
2024.0000,1.0045,171.5835
2025.0000,2.3389,132.8508
2026.0000,4.7447,102.8616
2027.0000,8.5234,79.6420
